# Uncomplete

In [4]:
from keras.layers import Subtract
from keras.models import Model

from utils import *

In [ ]:
VGG19ca

In [ ]:
Model.compile()

In [3]:
class DIDMDN:
    
    def __init__(self, img_shape=(128, 128, 3), g_filter=32, d_filter=64, lamdba_cycle=10.0, lambda_id=1.0):
        # Input shape
        self.img_rows, self.img_cols, self.channels = self.img_shape = img_shape

        # Calculate output shape of D (PatchGAN)
#         patch = int(self.img_rows / 2**4)
        patch = int(self.img_rows / 2**3)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.generator_filter, self.discriminator_filter = 32, 64

        # Loss weights
        self.lambda_cycle, self.lambda_id = lamdba_cycle, lambda_id

#         optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
    
        self.residue_net = Dense_rain_residual()
        self.residue_net.name = 'dense_rain_residual'
        self.vgg19_ca = VGG19ca()
        self.vgg19_ca.name = 'vgg19_ca'
        img_input = Input(shape=self.img_shape, name='input_img')
        label_input = Input(shape=self.img_shape, name='input_label')
        
        ### Using the Heavy rain Label (0) to get the residual ##
        output = self.residue_net(img_input, label_input)
        residue = Subtract()([img_input, output])
        label = self.vgg19_ca(residue)
        self.discriminator = Model(inputs=[img_input, label_input], outputs=[label])
        self.discriminator.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        

        ### Using the estimated resiudal to predict the label ##

        label = netG(residue)

        
        
        
        self.discriminator_A.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
        self.discriminator_B = self.build_discriminator()
        self.discriminator_B.name = 'discriminator_B'
        self.discriminator_B.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        
        self.dense_net = Dense_rain()
        self.dense_net.name = 'dense_rain'
        
        
        self.generator_A_to_B = self.build_generator()
        self.generator_A_to_B.name = 'generator_A_to_B'
        self.generator_B_to_A = self.build_generator()
        self.generator_B_to_A.name = 'generator_B_to_A'

        # Input images from both domains
        img_A = Input(shape=self.img_shape, name='input_A')
        img_B = Input(shape=self.img_shape, name='input_B')

        # Translate images to the other domain
        fake_B = self.generator_A_to_B(img_A)
        fake_A = self.generator_B_to_A(img_B)
        # Translate images back to original domain
        reconstr_A = self.generator_B_to_A(fake_B)
        reconstr_B = self.generator_A_to_B(fake_A)
        # Identity mapping of images
        img_A_id = self.generator_B_to_A(img_A)
        img_B_id = self.generator_A_to_B(img_B)

        # For the combined model we will only train the generators
        self.discriminator_A.trainable = False
        self.discriminator_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.discriminator_A(fake_A)
        valid_B = self.discriminator_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[valid_A, valid_B,
                                       reconstr_A, reconstr_B,
                                       img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[1, 1,
                                          self.lambda_cycle, self.lambda_cycle,
                                          self.lambda_id, self.lambda_id ],
                            optimizer='rmsprop')
    pass